# PreProcessing.ipynb

### This notebook pre-processes the Tweets for use in the SVM model.

Author: Erik Puijk <br>
Date  : February 17, 2022

In [9]:
import re
import json

In [10]:
def read_tweets(path):
    """ Read the Tweets from a given text file and return in JSON-format. """
    
    content = ""
    
    try:
        with open(path, 'r') as f:
            content = json.loads(f.read())
    except IOError:
        print("I/O error")
        
    print("Total Tweets read: %s" % (len(content)))

    return content

In [58]:
# Obtained from https://stackoverflow.com/questions/33404752/removing-emojis-from-a-string-in-python
def remove_emojis(tweet):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', tweet)

In [83]:
def remove_special_chars(tweet):
    # To lower case
    tweet = tweet.lower()
    
    # Replace URLs with 'URL'
    tweet = re.sub('http[A-Za-z0-9_:/.]+','', tweet)
    
    # Replace @mentions with 'NAME'
    tweet = re.sub('@[A-Za-z0-9_:/.]+', 'NAME', tweet)
    
    # Replace times with 'TIME' - still to check!
    tweet = re.sub('[0-9]+[:][0-9]+', 'TIME', tweet)
    tweet = re.sub('[0-9]+[.][0-9]+', 'TIME', tweet)
    
    # Replace numbers with 'NUMBER'
    
    
    # Remove lt, amp, etc!
    
    
    return re.sub(r'[^\w#]', ' ', tweet)

In [84]:
def pre_process(tweets):
    
    for tweet in tweets:
        tweet['text'] = remove_special_chars(tweet['text'])
        tweet['text'] = remove_emojis(tweet['text'])

    return tweets

In [85]:
tweets = read_tweets('source/tweets_all.txt')
tweets = pre_process(tweets)

for tweet in tweets:
    print(tweet['text'] + '\n')

Total Tweets read: 4667
stem net als mart de kruif op het cda  #ikstemcda #nudoorpakken 

de bedreigingen en intimidatie aan het adres van NAME laten zien dat haar en daarmee ons aller vrijheid op het spel staan   lale verdient nu onze steun  ik spreek me met ondertekening van dit manifest daarvoor uit  met vele anderen  teken ook   

van haga   het is nu of nooit voor onze vrijheid    via NAME

lijsttrekkersdebat rtl  wilders tegen de rest met  zwarte piet als minister voor cultuur   via NAME

ons #mkb gaat over de kop door onnavolgbare regels waardoor hardwerkende ondernemers geen steun krijgen  maar rutte en de vvd hebben wél goed voor het grootbedrijf gezorgd  nergens in europa kregen ze zoveel geld als hier  #fvd is er wél voor ons mkb   

 NAME gaat zonder mondkapje naar de supermarkt  dat willen wij ook wel  jammer dat hugo de lockdown  de avondklok en de zinloze mondkapjesplicht invoerde   wilt u net als hugo ook zonder mondkapje naar de supermarkt  stem dan 17 maart #fvd  

ee

we zijn live met de online talkshow #meerfeminisme    met kandidaat kamerleden  wetenschappers en activisten     kijk mee via 

de geheime groet van #fvd ers onder elkaar  

de tweede aflevering van de wij zijn bij1 podcast is nu te beluisteren  partijbestuurslid cheryl vliet gaat in gesprek met onze #3 NAME over de noodzaak van verandering      

de politieke correctheid leidt tot een cultuur waarin politici totaal niet het achterste van hun tong laten zien  daar doet #fvd niet aan mee  tweebeke probeert venijnig te doen  wij hebben het liever over onze plannen voor nederland  NAME  hier zit een echt mens   

vecht met isaura mee voor een inclusief nederland  stem NAME de kamer in   #denkanders #stemopeenvrouw #17maart2021 

jongeren en ondernemers kunnen heel veel voor elkaar betekenen  daarom wil de christenunie ze graag samenbrengen    hoe we dat willen doen  in het vandaag gelanceerde manifest jongeren  amp  ondernemers hebben we hiervoor tien ideeën uitgewerkt  

  vandaag om TIM

een woning is niet om winst op te maken  maar om in te wonen  de sp wil meer betaalbare woningen voor iedereen    NAME 

interview NAME over boek  dieren kunnen de pest krijgen  en dan   door NAME     het is van groot belang dat we de risico s op een volgende pandemie zo snel mogelijk verkleinen en daarom voorzichtiger omgaan met de natuur en minder dieren op een kluitje stoppen  

hoi zwevende kiezer  heb je een vraag over ons programma  ons kamerlid NAME zit klaar om je te helpen met je keuze  over treinen  klimaat  ongelijkheid of een ander onderwerp   stuur nu een appje naar onze  zweeflijn     #stemgroenlinks

wat doet u hier nog meneer rutte   #toeslagenaffaire #nederlandkiest #nosslotdebat #nosdebat #wilders #pvv #stempvv 

 de partij voor de dieren is klaar om mee te regeren  de vraag is of andere partijen klaar zijn om met ons te regeren       #planb NAME 

straks bij NAME  lijsttrekker NAME over ouderen die met minimale middelen moeten zien rond te komen en bespreekt hij wat 